# <font color="red"> MBA em IA e Big Data</font>
## <span style="color:red">Redes Neurais e Aprendizado Profundo</sp>

#Profa. Roseli Ap. Francelin Romero </br>

**Exemplo de Rede Neural com Aprendizado por Reforço**  para
resolver o Problema FrozenLake-v1


**Problema FrozenLake-v1**
O agente controla o movimento de um personagem em um grid (grade - forma matricial). Alguns blocos da grade podem ser percorridos, e outros fazem com que o agente caia na água. Além disso, a direção do movimento do agente é incerta e depende apenas parcialmente da direção escolhida. 
O agente é recompensado por encontrar um caminho percorrível até um bloco de meta. Um exemplo para uma grade 4x4 é mostrado abaixo:


SFFF       (S: starting point, safe) </br>
FHFH       (F: frozen surface, safe) </br>
FFFH       (H: hole, cai no buraco) </br>
HFFG       (G: goal, onde frisbee (disco) é localizado) </br>

O episódio termina se o agente encontra o alvo ou cai no buraco. Ao encontrar o alvo, ele recebe 1. Caso contrário, recebe 0.

No exemplo abaixo vamos trabalhar com um Grid de tamanho: 8x8 </br>

In [1]:
import gym
import numpy as np
import random
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow import keras

In [2]:
keras.utils.set_random_seed(7) 

AttributeError: module 'keras.api._v2.keras.utils' has no attribute 'set_random_seed'

In [ ]:
env = gym.make('FrozenLake-v1')  # definindo o ambiente

In [ ]:
tf.compat.v1.reset_default_graph()

In [ ]:
tf.compat.v1.disable_eager_execution()     # vai considerar vantagens de aceleracao
#Passo feed-forward na rede rasa usada para escolher ações
inputs1 = tf.compat.v1.placeholder(dtype=tf.float32,shape=(1,64))
W = tf.Variable(tf.random.uniform([64,4],0,0.01))
Qout = tf.matmul(inputs1,W)
predict = tf.argmax(Qout,1)

#Definição da função loss (perda) calculando o MSE entre o alvo e os valores Q preditos.
nextQ = tf.compat.v1.placeholder(dtype=tf.float32, shape=(1,4))
loss = tf.reduce_sum(tf.square(nextQ - Qout))
trainer = tf.compat.v1.train.GradientDescentOptimizer(learning_rate=0.1)
updateModel = trainer.minimize(loss)

In [ ]:
init = tf.compat.v1.initialize_all_variables()
 
# Set learning parameters
y = .99
e = 0.1
max_episode_length = 199
max_num_episodes = 2000
#Criando a lista que contem o total de recompensas e steps per episode
rList = []
#Armazena a media das recompensas da ultima 'window_size' dos episodios
average_ep_rewards = []
window_size = 50

with tf.compat.v1.Session() as sess:
    sess.run(init)
    for i in range(max_num_episodes):
        #Reseta o ambiente e inicializa com novas observacoes
        s = env.reset()
        rAll = 0
        d = False
        j = 0        
        #Definindo a Q-Network
        while j < max_episode_length:
            j+=1
            #Escolhe uma acao aleatoria (com e chance de acao aleatória) para a Q-network
            a,allQ = sess.run([predict,Qout],feed_dict={inputs1:np.identity(64)[s:s+1]})
            if np.random.rand(1) < e:
                a[0] = env.action_space.sample()
            #Vai para o novo estado do ambiente e pega a recompensa
            s1,r,d,_ = env.step(a[0])
            #Obtem os valores Q' alimentando o novo estado através da rede
            Q1 = sess.run(Qout,feed_dict={inputs1:np.identity(64)[s1:s1+1]})
            #Obtain maxQ' and set our target value for chosen action.
            maxQ1 = np.max(Q1)
            targetQ = allQ
            targetQ[0,a[0]] = r + y*maxQ1
            #Treina a rede usando target and predicted Q values
            _,W1 = sess.run([updateModel,W],feed_dict={inputs1:np.identity(64)[s:s+1],nextQ:targetQ})
            #Adding the returns
            rAll += r
            s = s1
            if d == True:
                #Reduz a chance de acao aleatoria durante o treinamento do modelo.
                e = 1./((i/50) + 10)
                break    
        rList.append(rAll)        
        if ((i+1) % window_size) == 0:          
          avg = np.mean(rList[-window_size:])
          print(f'Current episode no = {i}. Last {window_size} episodes mean reward = {avg}')
          average_ep_rewards.append(avg)
print("Training finished!")

In [ ]:
plt.plot(average_ep_rewards)